# 2. all manually small json files to one big json

In [20]:
import hashlib
import cv2

 
def get_md5(filename):
    result = None
    try:
        with open(filename,"rb") as f:
            bytess = f.read() # read file as bytes
            readable_hash = hashlib.md5(bytess).hexdigest();
            result = readable_hash
    except:
        pass
    return result


def get_shape(filename):
    image = cv2.imread(filename)
    r,c,channels = image.shape
    return r,c,channels


 

In [21]:
import os
import json


manual_name_to_category = {
    'RBC':'red blood cell',
    'Gametocyte':'gametocyte',
    'Ring':'ring',
    'Schizont':'schizont',
    'Trophozoite':'trophozoite'
}


m_json_list = []
f1 = 'Malariae//img/'
for f2 in os.listdir(f1):
    f3 = os.path.join(f1,f2)
    if f3.endswith('.json'):
        m_json_list.append(f3)
print('get small json files : {0}'.format(len( m_json_list )))

result = []

for f in m_json_list:
    with open(f, 'r') as fr:
        d = json.load(fr)
        tmp = {}
        tmp['image'] = {}
        img_f = f.replace('.json', '.jpg')
        tmp['image']['checksum'] = get_md5(img_f)
        tmp['image']['pathname'] = img_f
        r,c,channels = get_shape(img_f)
        tmp['image']['shape'] = {'r': r, 'c': c, 'channels': channels}
        
        tmp['objects'] = []
        for dd in d['shapes']:
            label = dd['label']
            r1,c1 = dd['points'][0]
            r2,c2 = dd['points'][1]
            r1 = int(r1)
            c1 = int(c1)
            r2 = int(r2)
            c2 = int(c2)
            tmp2 = {}
            tmp2['bounding_box'] = {}
            tmp2['bounding_box']['minimum'] = {'r': r1, 'c': c1}
            tmp2['bounding_box']['maximum'] = {'r': r2, 'c': c2}
            tmp2['category'] = manual_name_to_category[label]
            tmp['objects'].append(tmp2)
        
    result.append(tmp)

get small json files : 37


In [22]:
len(result)

37

In [23]:
import json

# Serializing json
json_object = json.dumps(result)

# Writing to sample.json
with open("test_vivax.json", "w") as outfile:
    outfile.write(json_object)

In [24]:
for d in result:
    objects = d['objects']
    for obj in objects:
        if 'bounding_box' not in obj:
            print('error', obj)

In [25]:
json_object

'[{"image": {"checksum": "a0303201edcae2fe5e2c48847b95338d", "pathname": "Malariae//img/1312132815-0001-G.jpg", "shape": {"r": 1944, "c": 2592, "channels": 3}}, "objects": [{"bounding_box": {"minimum": {"r": 1141, "c": 949}, "maximum": {"r": 1298, "c": 1080}}, "category": "gametocyte"}, {"bounding_box": {"minimum": {"r": 1220, "c": 328}, "maximum": {"r": 1360, "c": 463}}, "category": "red blood cell"}, {"bounding_box": {"minimum": {"r": 1379, "c": 949}, "maximum": {"r": 1506, "c": 1072}}, "category": "red blood cell"}, {"bounding_box": {"minimum": {"r": 1710, "c": 851}, "maximum": {"r": 1860, "c": 1001}}, "category": "red blood cell"}, {"bounding_box": {"minimum": {"r": 1327, "c": 759}, "maximum": {"r": 1456, "c": 888}}, "category": "red blood cell"}, {"bounding_box": {"minimum": {"r": 1306, "c": 630}, "maximum": {"r": 1445, "c": 753}}, "category": "red blood cell"}, {"bounding_box": {"minimum": {"r": 1179, "c": 753}, "maximum": {"r": 1306, "c": 888}}, "category": "red blood cell"}, {"

In [26]:
import os
import joblib as jl


dd = {}
for img in result:
    pathname = img['image']['pathname']
    basename = os.path.basename(pathname)
    dd[basename] = []
    for bbox in img['objects']:
        r1,c1 = bbox['bounding_box']['minimum']['r'],bbox['bounding_box']['minimum']['c']
        r2,c2 = bbox['bounding_box']['maximum']['r'],bbox['bounding_box']['maximum']['c']
        dd[basename].append(  (r1,c1,r2,c2)  )
    
jl.dump(dd, 'image_to_cells_labelme_vivax.jl')

['image_to_cells_labelme_vivax.jl']